# WHAT IS NLP?

NLP (Natural Language Processing) is a set of techniques for approaching text problems. This project will help 
you get started loading and cleaning the IMDB movie reviews, then applying a simple Bag of Words model to
get suprisingly accurate predictions of wheather a review is thumbs-up or thumbs-down.



## Reading the Data

This part of my project is not platform dependent and throughout the project I will be using
various Python modules for text processing, deep learning, random forest, and other applications.

Now lets get started by reading our tab-delimited file which contains 25,000 IMDB movie reviews, each with a positive or negativee sentiment label.

To do this , we can use the pandas package....

In [1]:
import pandas as pd

train = pd.read_csv(r"C:\Users\Pangloss\Downloads\word2vec-nlp-tutorial\labeledTrainData.tsv", header = 0, delimiter = "\t", quoting = 3)


Here, "header = 0" indicates that the first line of the file contains column names (Explicitly pass header=0 to be able to replace existing names), "delimiter = \t"indicates the field are separated by tabs and quoting  = 3 tells Python to ignore doubled quotes, otherwise you may encounter errors trying to read the file.

We can make sure that we read 25,000 rows and 3 columns as follows:

In [2]:
train.shape

train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

Now lets see the first movie review in the columns named "review"

In [3]:
print(train["review"][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

There are HTML tags "<br/>", abbreviations, punctuation - all common issues when processing text from online. Our next step is to tidy up the text for machine learning

# Data Cleaning and Text Preprocessing

Removing HTML Markup: The BeautifulSoup Package

First, I will remove the HTML tags. For this purpose, we'll use the BeautifulSoup library

In [4]:
from bs4 import BeautifulSoup

example1 = BeautifulSoup(train["review"][0])

print(train["review"][0])
print(example1.get_text())

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

Calling "get_text()" gives you the text of the review, without tags or markup. If you browse the BeauitulfulSoup documentation, you'll see that it's a very powerful library - more poewerful than we need for this dataset. However, it is not considered a reliable practice to remove markup using regular expressions, so even for an application as simple as this, it's usually best to use a package like BeautufulSoup

Dealling with Punctuation, Numbers and Stopwords: NLTK and regular expressions

When considering how to clean the text we should think about the data problem we are trying to solve. For many problems, it make sense to remove punctuations. On the other hand, in this case, we are tackling a sentiment analysis problem, and it is possble that "!!!' or ":-(" could carry sentiment, and should be treated as words. In this project, for simplicity, we will remove the punctuations altogether

To remove punctuation and numbers, we will use a package for dealing with reglar expressions, call "re"

In [5]:
import re

#Use regular expressions to do a find=and=replace

#re.sub(The Pattern to search for, The pattern to replace it with, The text to search)
letters_only = re.sub("[^a-zA-Z]", " ", example1.get_text())

print(letters_only)

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

The re.sub() statement above says, "Find anything that is not a lowercase letter(a-z) or an upper case letter (A-Z), and replace it with a space"

We'll also convert our reviews to lower case and split them into individual words (called "tokenization" in NLP lingo):

Tokenization is the process of replacing sensitive data with unique identification symbols that retain all the essential information about the data without compromising its security.

In this project we will use tokenization to break up the sentence into words and punctuation, and it is the first step to processing text.

In [6]:
lower_case = letters_only.lower()  # Convert to lower case

words = lower_case.split()         # Split into words 

Finally, we need to decide how to deal with frequently occuring words that don't carry much meaning. Such words are called 'stop words": in English they include words such as "a", "and", "is", and "the". Conveniently, there are Python packages that come with stop word list built in. Let's import a stop word list from the Python Natural Language Toolkit (NLTK). You'll need to install the library if you don't already have it on your computer

In [7]:
import nltk
#nltk.download()
from nltk.corpus import stopwords

print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Now lets remove stopwords from our movie review

In [8]:
words = [w for w in words if not w in stopwords.words("english")]
print(words)

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

This looks at each word in our "words" list , and discards anything that is found in the list of stop words.

There are many other things we could do to the data - For example, Porter Stemming and Lemmatizing (both available in NLTK) would allow us to treat "messages", "message", and "messaging" as the same word, which could certainly be useful. However, for simplicity of the project this will be the end of the first part  of the project.

Putting it all together

Now we have code to clean one review - but we need to clean 25,000 training reviews! To make our code reusable, let's create a function than can  be called many times:

In [9]:
def review_to_words( raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string ( a raw movie review), and 
    # the output is a single string( a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text()
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()
    
    #
    # 4. In Pythonm searching a set is much faster than serarching 
    #    a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))
    #
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string sparated by space,
    # and return the result.
    return( " ".join(meaningful_words ))

Two elements here are new: First, we converted the stop word list to a different data type, a set. This is for speed; since we'll be calling this
function tens of thousands of times, it needs to be fast, and searching sets
in Python is much faster than searching list.

Second, we joined the words back into one paragraph. This is to make the output easier to use in our Bag of Words.

Now let's loop through and clean all of the training set at once (this might take a few minutes depending on your computer)

In [10]:
## Get the number of reviews based on the dataframe column size


## Initialize an empty list to hold the clean reviews
#clean_train_reviews = []

## Loop over each review; create an index "i" that goes from 0 to the length
## of the movie review list
#for i in xrange( 0, num_reviews):
    # Call our functon for each one, and add the result to the list of 
    # clean reviews
  #  clean_train_reviews.append( review_to_words( train["review"][i] ) )

Sometimes it can be annoying to wait for a lengthy piece of code to run. It can be helpful to write code so that it gives status updates. To have Python print a status update after every 100 reviews that it processes, try adding a line or two to code above:

In [11]:
print("Cleaning and parsing the training set movie reviews...\n")

num_reviews = train["review"].size

clean_train_reviews = []

for x in range( 0, num_reviews ):
    # If the index is evenly divisible by 100, print a message
    if( (x+1)%100 == 0 ):
        print( "Review %d of %d\n" % ( x+1, num_reviews ))
    clean_train_reviews.append( review_to_words( train["review"][x] ) )

Cleaning and parsing the training set movie reviews...

Review 100 of 25000

Review 200 of 25000

Review 300 of 25000

Review 400 of 25000

Review 500 of 25000

Review 600 of 25000

Review 700 of 25000

Review 800 of 25000

Review 900 of 25000

Review 1000 of 25000

Review 1100 of 25000

Review 1200 of 25000

Review 1300 of 25000

Review 1400 of 25000

Review 1500 of 25000

Review 1600 of 25000

Review 1700 of 25000

Review 1800 of 25000

Review 1900 of 25000

Review 2000 of 25000

Review 2100 of 25000

Review 2200 of 25000

Review 2300 of 25000

Review 2400 of 25000

Review 2500 of 25000

Review 2600 of 25000

Review 2700 of 25000

Review 2800 of 25000

Review 2900 of 25000

Review 3000 of 25000

Review 3100 of 25000

Review 3200 of 25000

Review 3300 of 25000

Review 3400 of 25000

Review 3500 of 25000

Review 3600 of 25000

Review 3700 of 25000

Review 3800 of 25000

Review 3900 of 25000

Review 4000 of 25000

Review 4100 of 25000

Review 4200 of 25000

Review 4300 of 25000

Review 

# Creating Features from a Bag of Words (Using scikit-learn)

Now tha we have our training reviews tidied up, how do we convert them to some kind of numeric representation for machine learning? One common approach is called a Bag of Words. The Bag of Words model learns a vocabulary from all of the documents, then models each document by counting the number times each word apprears. For example, consider the following two sentences:

Sentence 1: "The cat sat on the hat"

Sentence 2: "The dog ate the cat and the hat"

From these two senetences, our vocabulary is as follows:

{the, cat, sat, on, dog, ate, and}

To get our bags of words, we count the number of times each word occurs in each sentence. In Sentence 1, "The" apprears twice. and "cat", "sat", "on" and "hat" each apprear once, so the feature vector for Sentence 1 is:

{the, cat, sat, on, hat, dog, ate, and}

Sentence 1: {2, 1, 1, 1, 1, 0, 0, 0}

Similarly, the features for Sentence 2 are: {3, 1, 0, 0, 1, 1, 1, 1}

In the IDMB data, we have a large number of reviews, which will give us a large vocabulary. To limit the size of the feature vectors, we should choose some maximum vocabulary size. Below, we use the 5000 most frequent words (remembering that stop words have already been removed).

We'll be using the feature_extraction module from scikit-learn to create bag-of-words features.

In [12]:
print("Creating the bag of words...\n")

from sklearn.feature_extraction.text import CountVectorizer

#Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool

vectorizer = CountVectorizer( analyzer = "word", 
                              tokenizer = None,
                              preprocessor = None,
                              stop_words = None,
                              max_features = 5000)

#fit_transform() does two function: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors.The input to fit_transform should be a list of
# strings

train_data_features = vectorizer.fit_transform(clean_train_reviews)

#Numpy arrays are easy to work with, so convert the result to an array

train_data_features = train_data_features.toarray()

Creating the bag of words...



In [13]:
train_data_features.shape

(25000, 5000)

It has 25,000 rows and 5,000 features (one or each vocabulary word).

Note that CountVectorizer comes with its own options to automatically do preprocessing m tokenization, and stop word removal == for each of these, instead of specifying "None", we coulod have used built-in method or specified our own function to use. However, we wanted to write our own function for data cleaning in this project to show you how it's done step by step.

Now that the Bag of Words model is trained, let's look at the vocabulary:

In [22]:
# Take a look at the words in the vocabulary

vocab_dict = vectorizer.vocabulary_.keys()

print(vocab_dict)

dict_keys(['stuff', 'going', 'moment', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'make', 'mind', 'whether', 'guilty', 'innocent', 'part', 'biography', 'feature', 'film', 'remember', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'anyway', 'hate', 'find', 'boring', 'may', 'call', 'making', 'movie', 'fans', 'would', 'say', 'made', 'true', 'nice', 'actual', 'bit', 'finally', 'starts', 'minutes', 'smooth', 'criminal', 'sequence', 'joe', 'convincing', 'powerful', 'drug', 'lord', 'wants', 'dead', 'beyond', 'plans', 'character', 'wanted', 'people', 'know', 'etc', 'hates', 'lots', 'things', 'turning', 'car', 'robot', 'whole', 'speed', 'demon', 'director', 'must', 'patience', 'cam

At this point, we have numeric training features from the Bag of Words and the original sentiment labels for each feature vector, so let's do some supervised learning! The Random Forest algorithm is included in scikit-learn (Random Forest uses many tree-based classifiers to make predictions, hence the "forest"). Below, we set the number of trees to 100 as a reasonable default value. More trees may (or may not) perform better, but will certainly take longer to run. Likewise, the more features you include for each review, the longer this will take.

In [15]:
print("Training the random forest...")

from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

Training the random forest...


All that remains is to run the trained Random Forest on our test set

Note that when we use the Bag of Words for the test set, we only call "transform", not "fit_transform" as we did for the training set. In machine learning, you shouldn't use the test set to fit your model, otherwise you run the risk of overfitting. For this reason, we keep the test set off-limits until we are ready to make predictions.

In [18]:
# Read the test data
test = pd.read_csv(r"C:\Users\Pangloss\Downloads\word2vec-nlp-tutorial\testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
test.shape

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( r"C:\Users\Pangloss\Downloads\word2vec-nlp-tutorial\Bag_of_Words_model.csv", index=False, quoting=3 )

Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000

